In [ ]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_f3fdba9d244c401187e179ad818538ca = 'https://s3.us.cloud-object-storage.appdomain.cloud'
else:
    endpoint_f3fdba9d244c401187e179ad818538ca = 'https://s3.private.us.cloud-object-storage.appdomain.cloud'

client_f3fdba9d244c401187e179ad818538ca = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='0v4d8hCD7rL44oC69GjDH3_pOadslP7Lnvq2zyPTqGiU',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_f3fdba9d244c401187e179ad818538ca)

body = client_f3fdba9d244c401187e179ad818538ca.get_object(Bucket='germancreditrisk-donotdelete-pr-ttjhbh8c1w4avk',Key='german_credit_data.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df1= pd.read_csv(body)
df1.head()


In [ ]:
# Delete unnecessary columns
del df1['Unnamed: 0']
df1.head()

In [ ]:
# Deal with NaN/Missing values
df1.info()

In [ ]:
# Create Dataframe to run regression to solve missing values
dfsolve = df1.dropna()
dfsolve.head()

In [ ]:
dfsolve = dfsolve.rename(columns={'Saving accounts': 'Saccounts', 'Checking account': 'Caccount'})
df1 = df1.rename(columns={'Saving accounts': 'Saccounts', 'Checking account': 'Caccount'})

In [ ]:
print(dfsolve["Saccounts"].unique())
print(dfsolve["Caccount"].unique())
print(dfsolve["Housing"].unique())

In [ ]:
dfsolve['saving']=dfsolve.Saccounts.map({'little':0, 'moderate':1, 'rich':2, 'quite rich':3})
dfsolve['checking']=dfsolve.Caccount.map({'little':0, 'moderate':1, 'rich':2,})
dfsolve['MorF']=dfsolve.Sex.map({'male':0, 'female':1})
dfsolve['Homepay']=dfsolve.Housing.map({'free':0, 'rent':1, 'own':2})
dfsolve.head()

In [ ]:
# Predict Savings Account amount
import numpy as np
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
x=dfsolve[['MorF', 'Job', 'Homepay',]]
y=dfsolve[['saving']]
regr = linear_model.LinearRegression()
regr.fit(x,y)
print(regr.coef_)

In [ ]:
x1=dfsolve[['MorF', 'Job', 'Homepay']]
y1=dfsolve[['checking']]
regr1 = linear_model.LinearRegression()
regr1.fit(x1,y1)
print(regr1.coef_)

In [ ]:
results = regr.score(x, y)
print(results)

In [ ]:
y.head(50)

In [ ]:
predict = regr.predict(x)
predict[0:50]

In [ ]:
import seaborn as sns
sns.heatmap(dfsolve.corr(), annot=True)

In [ ]:
df1['Saccounts'].hist()

In [ ]:
print(df1['Saccounts'].value_counts())
print(df1['Caccount'].value_counts())

In [ ]:
print (603/817)
print (103/817)
print (63/817)
print (48/817)

print(274/606)
print(269/606)
print(63/606)

In [ ]:
df1['Saccounts'] = df1['Saccounts'].fillna(pd.Series(np.random.choice(['little', 'quite rich', 'rich', 'moderate'], 
                                                      p=[0.738, 0.126, 0.077, .059], size=len(df1))))
df1.head()

In [ ]:
df1['Caccount'] = df1['Caccount'].fillna(pd.Series(np.random.choice(['little','rich', 'moderate'], 
                                                      p=[0.45, 0.44, 0.11], size=len(df1))))
df1.head()

In [ ]:
print(df1['Purpose'].unique())
print(df1['Risk'].unique())


In [ ]:
df1['Risk Outcome']=df1.Risk.map({'bad':0, 'good':1})
del df1['Risk']

In [ ]:
interval = (18, 25, 35, 60, 120)

Group = ['Student', 'Young', 'Adult', 'Senior']
df1["Age Group"] = pd.cut(df1.Age, interval, labels=Group)
del df1['Age']
df1.head()

In [ ]:
df2=pd.get_dummies(df1)
df2.head()

In [ ]:
x=df2.loc[:, df2.columns != 'Risk Outcome']
y=df2[['Risk Outcome']]

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2)

In [ ]:
df2.info()

In [ ]:
regr=linear_model.LogisticRegression()
regr.fit(xtrain,ytrain)
print(regr.coef_)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier().fit(xtrain, ytrain)
clfresults = clf.score(xtest, ytest)
print(clfresults)

In [ ]:
critclf = DecisionTreeClassifier(criterion="gini")
critclf = clf.fit(xtrain, ytrain)
critclfresults = clf.score(xtest,ytest)
print(critclfresults)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, confusion_matrix, classification_report, fbeta_score  
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV


seed = 7
models = []
models.append(('LGR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('RF', RandomForestClassifier()))
models.append(('SVM', SVC(gamma='auto')))
models.append(('XGBM', XGBClassifier()))
models.append(('LGBM', LGBMClassifier()))

In [ ]:
results = []
names = []
scoring = 'recall'

for name, model in models:
        kfold = KFold(n_splits=10, random_state=seed)
        cv_results = cross_val_score(model, xtrain, ytrain, cv=kfold, scoring=scoring)
        results.append(cv_results)
        names.append(name)
        msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
        print(msg)

fig = plt.figure(figsize=(11,6))
fig.suptitle('Algorithms Compare')
ax = fig.add_subplot(111)
green_diamond = dict(markerfacecolor='g', marker='D')
plt.boxplot(results, flierprops=green_diamond, patch_artist=True)
ax.set_xticklabels(names)
plt.show()